<a href="https://colab.research.google.com/github/NEERAJSHARMABSDU/neeraj/blob/master/Copy_of_Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from numpy import array
from pickle import dump
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [0]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

# load text
raw_text = load_doc('shakespeare.txt')
print(raw_text)

From fairest creatures we desire increase,
That thereby beauty's rose might never die,
But as the riper should by time decease,
His tender heir might bear his memory:
But thou contracted to thine own bright eyes,
Feed'st thy light's flame with self-substantial fuel,
Making a famine where abundance lies,
Thy self thy foe, to thy sweet self too cruel:
Thou that art now the world's fresh ornament,
And only herald to the gaudy spring,
Within thine own bud buriest thy content,
And tender churl mak'st waste in niggarding:
Pity the world, or else this glutton be,
To eat the world's due, by the grave and thee.


In [0]:
# clean
tokens = raw_text.split()
raw_text = ' '.join(tokens)

In [0]:
#organize into sequences of characters
length = 12
sequences = list()
for i in range(length, len(raw_text)):
    # select sequence of tokens
    seq = raw_text[i-length:i+1]
    # store
    sequences.append(seq)
    print('Total Sequences: %d' % len(sequences))

#Total Sequences: 597

# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()    
# save sequences to file
out_filename = 'char_sequences.txt'
save_doc(sequences, out_filename)

Total Sequences: 1
Total Sequences: 2
Total Sequences: 3
Total Sequences: 4
Total Sequences: 5
Total Sequences: 6
Total Sequences: 7
Total Sequences: 8
Total Sequences: 9
Total Sequences: 10
Total Sequences: 11
Total Sequences: 12
Total Sequences: 13
Total Sequences: 14
Total Sequences: 15
Total Sequences: 16
Total Sequences: 17
Total Sequences: 18
Total Sequences: 19
Total Sequences: 20
Total Sequences: 21
Total Sequences: 22
Total Sequences: 23
Total Sequences: 24
Total Sequences: 25
Total Sequences: 26
Total Sequences: 27
Total Sequences: 28
Total Sequences: 29
Total Sequences: 30
Total Sequences: 31
Total Sequences: 32
Total Sequences: 33
Total Sequences: 34
Total Sequences: 35
Total Sequences: 36
Total Sequences: 37
Total Sequences: 38
Total Sequences: 39
Total Sequences: 40
Total Sequences: 41
Total Sequences: 42
Total Sequences: 43
Total Sequences: 44
Total Sequences: 45
Total Sequences: 46
Total Sequences: 47
Total Sequences: 48
Total Sequences: 49
Total Sequences: 50
Total Seq

In [0]:
#load
in_filename = 'char_sequences.txt'
raw_text = load_doc(in_filename)
lines = raw_text.split('\n')

chars = sorted(list(set(raw_text)))
mapping = dict((c, i) for i, c in enumerate(chars))

In [0]:
sequences = list()
for line in lines:
    #integer encode line
    encoded_seq = [mapping[char] for char in line]
    #store
    sequences.append(encoded_seq)

#vocabulary size
vocab_size = len(mapping)
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 37


In [0]:
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]

sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = array(sequences)
y = to_categorical(y, num_classes=vocab_size)
X.shape

(597, 12, 37)

In [0]:
# define the model
def define_model(X):
    model = Sequential()
    model.add(LSTM(75, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dense(vocab_size, activation='softmax'))
    # compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # summarize defined model
    model.summary()
   # plot_model(model, to_file='model.png', show_shapes=True)
    return model

In [0]:
# define model
model = define_model(X)
# fit model
model.fit(X, y, epochs=100, verbose=2)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 75)                33900     
_________________________________________________________________
dense_1 (Dense)              (None, 37)                2812      
Total params: 36,712
Trainable params: 36,712
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
 - 1s - loss: 3.5488 - acc: 0.1240
Epoch 2/100
 - 0s - loss: 3.2005 - acc: 0.1742
Epoch 3/100
 - 0s - loss: 3.0050 - acc: 0.1742
Epoch 4/100
 - 0s - loss: 2.9713 - acc: 0.1742
Epoch 5/100
 - 0s - loss: 2.9559 - acc: 0.1742
Epoch 6/100
 - 0s - loss: 2.9461 - acc: 0.1742
Epoch 7/100
 - 0s - loss: 2.9363 - acc: 0.1742
Epoch 8/100
 - 0s - loss: 2.9322 - acc: 0.1742
Epoch 9/100
 - 0s - loss: 2.9173 - acc

In [0]:
# save the model to file
model.save('model.h5')
# save the mapping
dump(mapping, open('mapping.pkl', 'wb'))

In [0]:
from pickle import load
from keras.models import load_model
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

# load the model
model = load_model('model.h5')
# load the mapping
mapping = load(open('mapping.pkl', 'rb'))

In [0]:
# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    in_text = seed_text
    len(in_text)
    # generate a fixed number of characters
    for _ in range(n_chars):
        # encode the characters as integers
        encoded = [mapping[char] for char in in_text]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # one hot encode
        encoded = to_categorical(encoded, num_classes=len(mapping))
        encoded.shape
        #encoded = encoded.reshape( 1,encoded.shape[0], encoded.shape[1])
        # predict character
        yhat = model.predict_classes(encoded, verbose=0)
        # reverse map integer to character
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
            # append to input
        in_text += out_char
    return in_text

In [0]:
# test start of rhyme
print(generate_seq(model, mapping, 12, 'From fairest', 70))
# test mid-line
print(generate_seq(model, mapping, 12, 'Making a famine', 70))
# test not in original
print(generate_seq(model, mapping, 12, 'hello worl', 70))

From fairest creatures we desire increase, That thereby beauty's rose might never 
Making a famine where abundance lies, Thy self thy foe, to thy sweet self too cruel: 
hello world bud sesttin tonef oo buryy oest to ty reee, The thearey bp tae ses m
